In [1]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
spark = SparkSession.builder.appName("1211").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/21 04:55:15 WARN Utils: Your hostname, de24, resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp0s3)
25/07/21 04:55:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/21 04:55:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/21 04:55:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/21 04:55:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
'''
Table: Queries

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| query_name  | varchar |
| result      | varchar |
| position    | int     |
| rating      | int     |
+-------------+---------+
This table may have duplicate rows.
This table contains information collected from some queries on a database.
The position column has a value from 1 to 500.
The rating column has a value from 1 to 5. Query with rating less than 3 is a poor query.
 

We define query quality as:

The average of the ratio between query rating and its position.

We also define poor query percentage as:

The percentage of all queries with rating less than 3.

Write a solution to find each query_name, the quality and poor_query_percentage.

Both quality and poor_query_percentage should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Queries table:
+------------+-------------------+----------+--------+
| query_name | result            | position | rating |
+------------+-------------------+----------+--------+
| Dog        | Golden Retriever  | 1        | 5      |
| Dog        | German Shepherd   | 2        | 5      |
| Dog        | Mule              | 200      | 1      |
| Cat        | Shirazi           | 5        | 2      |
| Cat        | Siamese           | 3        | 3      |
| Cat        | Sphynx            | 7        | 4      |
+------------+-------------------+----------+--------+
Output: 
+------------+---------+-----------------------+
| query_name | quality | poor_query_percentage |
+------------+---------+-----------------------+
| Dog        | 2.50    | 33.33                 |
| Cat        | 0.66    | 33.33                 |
+------------+---------+-----------------------+
Explanation: 
Dog queries quality is ((5 / 1) + (5 / 2) + (1 / 200)) / 3 = 2.50
Dog queries poor_ query_percentage is (1 / 3) * 100 = 33.33

Cat queries quality equals ((2 / 5) + (3 / 3) + (4 / 7)) / 3 = 0.66
Cat queries poor_ query_percentage is (1 / 3) * 100 = 33.33

'''

In [5]:
data = [
('Dog','Golden Retriever',1  ,5),
('Dog','German Shepherd' ,2  ,5),
('Dog','Mule'            ,200,1),
('Cat','Shirazi'         ,5  ,2),
('Cat','Siamese'         ,3  ,3),
('Cat','Sphynx'          ,7  ,4)
]
schema = ['query_name','result','position','rating']

In [8]:
t1_df = spark.createDataFrame(data=data,schema=schema)
t1_df.show()

+----------+----------------+--------+------+
|query_name|          result|position|rating|
+----------+----------------+--------+------+
|       Dog|Golden Retriever|       1|     5|
|       Dog| German Shepherd|       2|     5|
|       Dog|            Mule|     200|     1|
|       Cat|         Shirazi|       5|     2|
|       Cat|         Siamese|       3|     3|
|       Cat|          Sphynx|       7|     4|
+----------+----------------+--------+------+



In [24]:
t1_df.groupBy(F.col("query_name"))\
     .agg(
         F.round(F.avg(F.col("rating") / F.col("position")),2).alias("quality"),
         F.round(F.sum( F.when( F.col("rating") < 3, 1 ).otherwise(0) )/F.count("*"),2).alias("poor_query_percentage")
         )\
     .show()

[Stage 18:=============================>                            (3 + 3) / 6]

+----------+-------+---------------------+
|query_name|quality|poor_query_percentage|
+----------+-------+---------------------+
|       Dog|    2.5|                 0.33|
|       Cat|   0.66|                 0.33|
+----------+-------+---------------------+



## SQL Solution

<pre>
SELECT query_name, 
       ROUND(AVG(rating/position),2) as quality, 
       ROUND(COUNT(CASE WHEN rating < 3 THEN 1 ELSE 0 END)/COUNT(*)*100,2) as poor_query_percentage
FROM Queries
GROUP BY query_name
</pre>